In [1]:
from main import load_and_preprocess_data

df = load_and_preprocess_data(0.1)

/Users/heifen/Documents/studie/machinelearning/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading and preprocessing data...
Class distribution:
is_fraud
0    1289169
1       7506
Name: count, dtype: int64
Fraud percentage: 0.58%
Sampled 90 users (10.0% of original dataset)
Sampled dataset shape: (119270, 34)


In [2]:
from model import load_lstm_model
from predict import predict_and_analyze
from main import SEQUENCE_LENGTH

sample_cc_num = df['cc_num'].iloc[0]
user_transactions = df[df['cc_num'] == sample_cc_num]

last_transaction = user_transactions.iloc[-1].to_dict()

lstm_model, processor = load_lstm_model()

results = predict_and_analyze(
    lstm_model,
    user_transactions.iloc[:-1],  # Use all but last transaction
    SEQUENCE_LENGTH,
    processor,
    actual_transaction=last_transaction
)

print("\nPredicted most likely values for next transaction:")
print("---------------------------------------------")
for feature, value in results['prediction']['most_likely_values'].items():
    print(f"  {feature}: {value}")
            

print('\nReal values:')
print("---------------------------------------------")
for feature in results['prediction']['most_likely_values'].keys():
    print(f"  {feature}: {last_transaction[feature]}")


Loading model from models/lstm_transaction_model.h5...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step

Predicted most likely values for next transaction:
---------------------------------------------
  amt: {'lower_bound': np.float64(66.19), 'upper_bound': np.float64(68.94540000000008), 'avg': np.float64(67.56770000000003)}
  lat: {'lower_bound': np.float64(43.0048), 'upper_bound': np.float64(44.3481), 'avg': np.float64(43.67645)}
  long: {'lower_bound': np.float64(-108.8964), 'upper_bound': np.float64(-101.2332), 'avg': np.float64(-105.06479999999999)}
  merch_lat: {'lower_bound': np.float64(43.19807706666666), 'upper_bound': np.float64(43.7548648), 'avg': np.float64(43.47647093333333)}
  merch_long: {'lower_bound': np.float64(-116.73911386666667), 'upper_bound': np.float64(-108.72713016666667), 'avg': np.float64(-112.73312201666667)}
  hour: {'avg': np.int64(13)}
  day: {'avg': np.int64(20)}
  month: {'avg': np.int64(6)}
  dayofweek: {'avg': np.int64(6)}
  merchant_encoded: {'avg': np.int64(